In [ ]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

In [ ]:
import zcode
import zcode.math as zmath
import zcode.plot as zplot

In [ ]:
gsmf = holo.sam.GSMF_Schechter()        # Galaxy Stellar-Mass Function (GSMF)
gpf = holo.sam.GPF_Power_Law()          # Galaxy Pair Fraction         (GPF)
gmt = holo.sam.GMT_Power_Law()          # Galaxy Merger Time           (GMT)
mmbulge = holo.relations.MMBulge_Standard()     # M-MBulge Relation            (MMB)
# shape = (150, 151, 152)
shape = (30, 31, 32)
# shape = None

sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge, shape=shape)

In [ ]:
fobs = utils.nyquist_freqs(10.0*YR, 0.1*YR)
# fobs = utils.nyquist_freqs(10.0*YR, 1.0*YR)
# fobs = kale.utils.spacing(fobs, scale='log', num=fobs.size)

In [ ]:
NUM_REALS = 30

In [ ]:
gwb_smooth = sam.gwb(fobs, realize=False)
gwb_rough = sam.gwb(fobs, realize=NUM_REALS)

In [ ]:
gwb = np.zeros((fobs.size-1, NUM_REALS))
gwf = np.zeros((fobs.size-1, NUM_REALS))
for ii in utils.tqdm(range(NUM_REALS)):
    gwf_freqs, gwf[:, ii], gwb[:, ii] = holo.sam.sampled_gws_from_sam(
        sam, fobs=fobs, hard=holo.evolution.Hard_GW, cut_below_mass=1e6*MSOL, limit_merger_time=None,
        sample_threshold=10, poisson_inside=True, poisson_outside=True,
    )
    # gwb[:, ii] = np.sqrt(_gwb[:]**2 + gwf**2)
    # break

In [ ]:
fig, ax = plot.figax()

def plot_gwb(xx, yy, **kwargs):
    cc, = ax.plot(xx, np.median(yy, axis=-1), ls='--', **kwargs)
    cc = cc.get_color()
    temp = np.percentile(yy, [25, 75], axis=-1)
    ax.fill_between(xx, *temp, color=cc, alpha=0.25)    
    return

xx = kale.utils.midpoints(fobs) * YR   # [1/sec] ==> [1/yr]

# amp = 10e-16
# yy = amp * np.power(xx, -2/3)
# ax.plot(xx, yy, 'k--', alpha=0.25)

ax.plot(xx, gwb_smooth, 'b:')
plot_gwb(xx, gwb_rough, label='roughed')
if np.ndim(gwb) == 1:
    ax.plot(xx, gwb, 'k-', alpha=0.25)
else:
    plot_gwb(xx, gwb, label='sampled')
    plot_gwb(xx, np.sqrt(gwb**2 + gwf**2), label='sampled')

plt.legend()
plt.show()
# fig.savefig('temp.png')

# Manual

In [ ]:
gsmf = holo.sam.GSMF_Schechter()        # Galaxy Stellar-Mass Function (GSMF)
gpf = holo.sam.GPF_Power_Law()          # Galaxy Pair Fraction         (GPF)
gmt = holo.sam.GMT_Power_Law()          # Galaxy Merger Time           (GMT)
mmbulge = holo.relations.MMBulge_Standard()     # M-MBulge Relation            (MMB)
hard = holo.evolution.Hard_GW
# shape = (150, 151, 152)
shape = (30, 31, 32)
# shape = (30, 31, 300)
# shape = None

sam = holo.sam.Semi_Analytic_Model(
        gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge, shape=shape,
        redz=[1e-6, 6.0, 32]
)
print(f"{sam.shape=}")

fobs = utils.nyquist_freqs(10.0*YR, 0.1*YR)

vals, weights, edges, dens, mass = holo.sam.sample_sam_with_hardening(
        sam, hard, fobs=fobs, cut_below_mass=None, limit_merger_time=None,
        sample_threshold=10.0, poisson_inside=True, poisson_outside=True,
)
print(f"{weights.size=:.4e}, {weights.sum()=:.8e}")

In [ ]:
gwb_smooth = sam.gwb(fobs, realize=False)
gwb_rough = sam.gwb(fobs, realize=NUM_REALS)

In [ ]:
use_weights = np.copy(weights)
use_vals = np.copy(vals)
print(kale.utils.stats_str(use_weights))

# sel = (use_vals[2] < 0.5)
# use_vals[2][sel] *= 2.0

sel = slice(None)
# sel = (vals[2] > 0.5)
# sel = (use_weights > 1.0)
# sel = (use_weights < 1.0)
# sel = (use_weights != 1.0)
# sel = ~sel

use_weights = use_weights[sel]
# use_weights = np.random.poisson(use_weights)
print(kale.utils.stats_str(use_weights))

# print(kale.utils.stats_str(use_weights[sel]))
# print(kale.utils.stats_str(use_weights[~sel]))
# print(f"weights: {zmath.frac_str(sel)}")

gff, gwf, gwb = holo.sam._gws_from_samples(use_vals[:, sel], use_weights, fobs)
fig, ax = plot.figax()
ff = kale.utils.midpoints(fobs)
ax.plot(ff, gwb)
ax.plot(gff, gwf, 'rx', alpha=0.5)

ax.plot(ff, gwb_smooth, 'b:')
plot_gwb(ff, gwb_rough, label='roughed')

plt.show()

# Compare Properties of Sampled Population to pure-SAM

In [ ]:
NUM_REALS = 10
vals = []
weights = []
for ii in holo.utils.tqdm(range(NUM_REALS)):
    _vals, _weights, edges, dens, mass = holo.sam.sample_sam_with_hardening(
            sam, holo.evolution.Hard_GW, fobs=fobs,
            sample_threshold=5.0, cut_below_mass=None, limit_merger_time=None,
    )
    vals.append(_vals)
    weights.append(_weights)

## Number of Sources vs. Frequency

In [ ]:
num_fobs = fobs.size
MASS_MIN = 1.0e8 * MSOL
# MASS_MIN = 0.0/
num = np.zeros((num_fobs, NUM_REALS))

for ii in holo.utils.tqdm(range(NUM_REALS)):
    fo = vals[ii][-1]
    idx = np.digitize(fo, fobs) - 1
    for jj in range(num_fobs):
        sel = (idx == jj) & (vals[ii][0] > MASS_MIN)
        num[jj, ii] = np.sum(weights[ii][sel])
        
sel = (kale.utils.midpoints(edges[0]) > MASS_MIN)
idx = list(np.arange(mass.ndim))
idx.pop(mass.ndim - 1)
sam_num = np.sum(mass[sel], axis=tuple(idx))

In [ ]:
fig, ax = plot.figax()
xx = fobs*YR
# ax.plot(xx, num)
ax.plot(kale.utils.midpoints(xx), num.mean(axis=-1)[:-1], 'b-')
ax.plot(kale.utils.midpoints(xx), sam_num, 'r--')

aa, bb = plot._get_hist_steps(xx, sam_num)
ax.plot(aa, bb, 'r:')
# ax.set(xlim=[0.95e-1, 2.5e-1], ylim=[2e9, 4e10])
plt.show()

# Chirp-mass Distribution

In [ ]:
mchirp = []
for ii in range(NUM_REALS):
    vv = vals[ii]
    mc = utils.m1m2_from_mtmr(vv[0], vv[1])
    mc = utils.chirp_mass(*mc)
    print(mc.shape, utils.stats(mc/MSOL))
    break

## Distance/Redshift Distribution

In [ ]:
gsmf = holo.sam.GSMF_Schechter()        # Galaxy Stellar-Mass Function (GSMF)
gpf = holo.sam.GPF_Power_Law()          # Galaxy Pair Fraction         (GPF)
gmt = holo.sam.GMT_Power_Law()          # Galaxy Merger Time           (GMT)
mmbulge = holo.relations.MMBulge_Standard()     # M-MBulge Relation            (MMB)
hard = holo.evolution.Hard_GW
# shape = (150, 151, 152)
shape = (30, 31, 32)
# shape = (30, 31, 300)
# shape = None

sam = holo.sam.Semi_Analytic_Model(
        gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge, shape=shape,
        redz=[0.01, 6.0, 32]
)
print(f"{sam.shape=}")

fobs = utils.nyquist_freqs(10.0*YR, 0.1*YR)

In [ ]:
z = 1.0e-3

zz = np.linspace(0.0, z, 100)
# dv = cosmo.differential_comoving_volume(zz) * 4*np.pi
dv = cosmo.dVcdz(zz, cgs=False)
v1 = np.sum(0.5 * (dv[1:] + dv[:-1]) * np.diff(zz))
v2 = cosmo.comoving_volume(zz)
print(v1, v2[-1], v1/v2[-1])
aa = v2[1]
bb = 0.5 * (dv[0] + dv[1]) * zz[1]
print(aa, bb, aa/bb)

In [ ]:
cosmo.dcom_to_z(dc)

In [ ]:
vals, weights, edges, dens, mass = holo.sam.sample_sam_with_hardening(
        sam, hard, fobs=fobs, cut_below_mass=None, limit_merger_time=None,
        sample_threshold=10.0, poisson_inside=True, poisson_outside=True,
)
print(f"{weights.size=:.4e}, {weights.sum()=:.8e}")

In [ ]:
NBINS = 30
dc = cosmo.comoving_distance(vals[2]).to('Mpc').value
nsamp = weights.sum()
dcmax = dc.max()
dens = nsamp / ((4.0*np.pi*dcmax**3)/3.0)

fig, axes = plot.figax(nrows=2)

ax = axes[0]
xx = vals[2]
bins = kale.utils.spacing(xx, 'log', NBINS)
ax.hist(xx, bins=bins, histtype='step', weights=weights)
ax.hist(xx, bins=sam.edges[2], histtype='step', weights=weights)

ax = axes[1]
xx = dc
bins = kale.utils.spacing(xx, 'log', NBINS)
ax.hist(xx, bins=bins, histtype='step', weights=weights)
yy = ((4.0*np.pi*bins**3)/3.0) * dens
ax.plot(bins, yy, 'k--')

plt.show()

In [ ]:
NBINS = 10
dc = cosmo.comoving_distance(vals[2]).to('Mpc').value
nsamp = weights.sum()
dcmax = dc.max()
dens = nsamp / ((4.0*np.pi*dcmax**3)/3.0)

fig, axes = plot.figax(nrows=2)

ax = axes[0]
xx = vals[2]
bins = kale.utils.spacing(xx, 'log', NBINS)
ax.hist(xx, bins=bins, histtype='step', weights=weights)
ax.hist(xx, bins=sam.edges[2], histtype='step', weights=weights)

ax = axes[1]
xx = dc
bins = kale.utils.spacing(xx, 'log', NBINS)
ax.hist(xx, bins=bins, histtype='step', weights=weights)
yy = ((4.0*np.pi*bins**3)/3.0) * dens
ax.plot(bins, yy, 'k--')

plt.show()